## Setup configuration for our validation

In [1]:
HD_OUTPUT_FILENAME = 'hashdice_validation.txt'
PREVIOUS_TIMESTEP_COUNT = 4
IMPORT_COUNT = 100000 + PREVIOUS_TIMESTEP_COUNT

## Generate deterministic dataset

In [2]:
import hmac
import hashlib

def hashdice(secret_key, message):
    hmac_sha512 = hmac.new(key=secret_key.encode(), msg=message.encode(), digestmod=hashlib.sha512).hexdigest()
    lucky = int(hmac_sha512[0:5], 16)
    return lucky % 100000


with open(HD_OUTPUT_FILENAME, 'w') as f:

    for i in range(IMPORT_COUNT + 1):
        secret_key = '4671f003b18db91be8f1570c7f681ad9e97026c5f34d89a733f48ab81eb8d592'
        message = f'5533idsczGPRYacG5N15jfJ6nm5:{i}'
        result = hashdice(secret_key, message)
        rs = str(result)
        f.write(f'{rs + str(i%10000)}\n')

## Convert to input and target data

In [3]:
import numpy as np

# convert the sequence of generated numbers to 4 inputs and one output
def strided(a, L):
    shp = a.shape
    s  = a.strides
    nd0 = shp[0]-L+1
    shp_in = (nd0,L)+shp[1:]
    strd_in = (s[0],) + s
    return np.lib.stride_tricks.as_strided(a, shape=shp_in, strides=strd_in)

df = np.genfromtxt(HD_OUTPUT_FILENAME,delimiter='\n',dtype='uint64')
TOTAL_DATA_NUM = IMPORT_COUNT-PREVIOUS_TIMESTEP_COUNT

# calculates how many bits are in the output.
BIT_WIDTH = np.ceil(np.log2(np.amax(df))).astype(int)
print(BIT_WIDTH)
# convert the generated numbers to binary sequences
df_as_bits =(df[:,None] & (1 << np.arange(BIT_WIDTH,dtype='uint64')) > 0).astype(int)
df_as_frames = strided(df_as_bits, PREVIOUS_TIMESTEP_COUNT+1)

#indicies = np.arange(TOTAL_DATA_NUM,dtype='uint64')
#print(indicies)
#np.random.shuffle(indicies)
#print(indicies)
#df_as_frames=df_as_frames[indicies]

# convert the data into inputs and outputs
y = df_as_frames[:,-1,:]
X = df_as_frames[:,:-1,]
X = X.reshape([X.shape[0], X.shape[1]*X.shape[2]])

print(np.shape(X), np.shape(y))


print(X[0])
print(y[0])

30
(100001, 120) (100001, 30)
[0 1 0 0 1 1 1 1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0
 0 0 1 1 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 0 1 1 1 0 0 1 1 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0]
[0 1 0 1 0 0 1 0 1 1 0 0 0 0 1 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0]


## Load pre-trained model

In [4]:
from tensorflow.keras.models import load_model
model = load_model('hashdice_model.h5')

## Validate bitwise accuracy

In [5]:
results = model.evaluate(X, y, batch_size=256)
print("test loss: %f, test acc: %s" % tuple(results))

391/391 [==============================] - 2s 4ms/step - loss: 0.5893 - binary_accuracy: 0.6081
test loss: 0.589338, test acc: 0.608125627040863


In [13]:
import tensorflow as tf
index = 4
inputTest=np.reshape(X[index],(1,120))
predict = model.predict(inputTest)


orignalResult = ', '.join([str(x) for x in predict[0]])
print(f'Original Result {orignalResult}')
orignalResult = ', '.join([str(x) for x in predict[0][::-1]])
print(f'-------------------------')
print(f'Original Result {orignalResult}')
npWhere=np.where(predict[0] > 0.5, 1, 0)
print(f'reverse correct output------------{y[index]}')
print(f'predicted output reverse----------{npWhere}')
o = y[index][::-1]
print(f'correct output--------------------{o}')
npWhere = npWhere[::-1]
print(f'predicted output------------------{npWhere}')
tfround = tf.round(predict[0])
#print(f'tfround {tfround}')

output = sum(bit * 2**(len(o)-i-1) for i, bit in enumerate(o))
print(f'correct output--------------------{output}')
result = np.array(tfround[::-1])
result = sum(bit * 2**(len(result)-i-1) for i, bit in enumerate(result))
print(f'predicted output------------------{result}')

1/1 [==============================] - 0s 48ms/step
Original Result 2.1703588e-08, 0.7316017, 0.81323016, 0.9431137, 0.4095405, 0.34946904, 0.35475978, 0.4875862, 0.29990268, 0.2936106, 0.46594527, 0.6648567, 0.64295185, 0.19875206, 0.48983806, 0.5256254, 0.50200075, 0.655248, 0.25556362, 0.84801924, 0.11043845, 0.016831614, 0.035998885, 0.20185827, 0.002463287, 0.0019066184, 0.005732572, 1.1864399e-06, 1.505222e-06, 1.3652289e-05
-------------------------
Original Result 1.3652289e-05, 1.505222e-06, 1.1864399e-06, 0.005732572, 0.0019066184, 0.002463287, 0.20185827, 0.035998885, 0.016831614, 0.11043845, 0.84801924, 0.25556362, 0.655248, 0.50200075, 0.5256254, 0.48983806, 0.19875206, 0.64295185, 0.6648567, 0.46594527, 0.2936106, 0.29990268, 0.4875862, 0.35475978, 0.34946904, 0.4095405, 0.9431137, 0.81323016, 0.7316017, 2.1703588e-08
reverse correct output------------[0 1 1 1 0 0 0 1 0 1 0 1 1 0 1 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0]
predicted output reverse----------[0 1 1 1 0 0 0 0 0 0 0 1 1

In [54]:
#000000000011010100001101001010
#000000000001101001101101001110